In [60]:
%matplotlib inline
#!/usr/bin/env python
# -*- coding:utf-8 -*-
__author__ = "yan bin"
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.pipeline import Pipeline,PipelineModel
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import *
from pyspark.sql.functions import *  
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit,CrossValidator
from pandas import *
import matplotlib as mpl
import time
from numpy import arange,logspace
import sys
mpl.use("Agg")
import matplotlib.pyplot as plt

def myInt(x):
    try:
        return int(x)
    except:
        return np.nan
    
def map_extract(element):
    file_path, content = element
    year = file_path[-8:-4]
    return [(year, i) for i in content.split("\r\n") if i]

class spark:
    __spark = 0
    __sc = 0


    def __init__(self, master="local", appName="sparkDemo"):
        conf = (SparkConf().setAppName(appName)
                .setMaster(master))
        self.__spark = (SparkSession.builder
                .config(conf=conf)
                .enableHiveSupport()
                .getOrCreate())
        self.__sc = self.__spark.sparkContext


    def getContext(self):
        return (self.__spark, self.__sc)


    def getJson(self, inputPath):
        spark, sc = self.getContext()
        rawData = spark.read.json(inputPath)
        ratingDf = (rawData.select(rawData.reviewerID.alias("user"),
                                   rawData.asin.alias("item"),
                                   rawData.overall.alias("rating")))
        return ratingDf
    
    def getText(self, inputPath):
        spark, sc = self.getContext()
        rawData = sc.textFile(inputPath)
        rawData = (rawData.map(lambda x:x.split("::"))
                            .map(lambda x:(myInt(x[0]),myInt(x[1]),myInt(x[2]),myInt(x[3]))))
        ratingDf =spark.createDataFrame(rawData,("user","item","rating","timestamp"))
        return ratingDf.na.drop() 
    
                 
    def getDoc(self,inputPath):
        spark, sc = self.getContext()
        res=sc.wholeTextFiles(inputPath,minPartitions=40)
        return res
    
        
        
if __name__ == "__main__":
    start = time.time()
    s = spark(appName="test",master="local[4]")
    spark, sc = s.getContext()
    inputPath="data/names"
    res = s.getDoc(inputPath)
    from operator import add
#     newresw = res.map(lambda x:[(x[0][-8:-4],int(i.split(",")[-1])) for i in x[1].split("\r\n") if i])\
#     .flatMap(lambda x:x).reduceByKey(add).sortByKey()
    newres = res.map(lambda x: (x[0][-8:-4],[int(i.split(",")[-1]) for i in x[1].split("\r\n") if i]))\
    .flatMapValues(lambda x:x).reduceByKey(add).sortByKey()
#     print (newres.take(5))
#     print (newresw.take(5))
    newres.collect()
    
    sc.stop()
    spark.stop()


In [61]:
import pandas as pd
data = pd.DataFrame.from_records(newres.collect(), columns=['year', 'birth'])\
             .sort(columns=['year'], ascending=True)
ax = data.plot(x=['year'], y=['birth'], 
                    figsize=(20, 6), 
                    title='US Baby Birth Data from 1897 to 2014', 
                    linewidth=3)
ax.set_axis_bgcolor('white')
ax.grid(color='gray', alpha=0.2, axis='y')
sc.stop()
spark.stop()


AttributeError: 'NoneType' object has no attribute 'setCallSite'